In [1]:
import sys
sys.path.append('/home/onyxia/work/libsigma')

In [18]:
import os
import read_and_write as rw
from osgeo import gdal
import geopandas as gpd

In [19]:
test=gpd.read_file('/home/onyxia/work/data/project/emprise_etude.shp')
test

,MINX,MINY,MAXX,MAXY,CNTX,CNTY,AREA,PERIM,HEIGHT,WIDTH,...,MINY_2,MAXX_2,MAXY_2,CNTX_2,CNTY_2,AREA_2,PERIM_2,HEIGHT_2,WIDTH_2,geometry
0,501127.9697,6.238861e+06,610073.1663,6.314464e+06,555600.568,6.276662e+06,8.236616e+09,369096.9934,75603.3001,108945.1966,...,4790220.0,409800.0,4900020.0,354900.0,4845120.0,1.205604e+10,439200.0,109800.0,109800.0,"POLYGON ((501127.97 6240794.776, 501127.97 631..."


In [26]:
test['MINY_2']

0    4790220.0
Name: MINY_2, dtype: float64

# Partie production image

L'idée ça peut être de construire une image comme dans le td opération sur raster en lui ajoutant à chaque fois une nouvelle bande d'une des dates

Donc d'abord charger toutes les images dans un tableau avec 60 bandes

In [11]:
#Façon potentiel pour clip un raster grâce à l'emprise

import geopandas as gpd
from osgeo import gdal

# Charger le vecteur avec Geopandas
gdf = gpd.read_file("/home/onyxia/work/data/project/emprise_etude.shp").to_crs("EPSG:2154")

# Extraire le GeoJSON sous forme de string
geojson_str = gdf.to_json()

# Appliquer le clip avec GDAL
resolution = 10  # Résolution (10 m)
gdal.Warp(
    "/home/onyxia/work/test.tif",     #Chemin fichier en sortie
    "/home/onyxia/work/data/images/SENTINEL2B_20220125-105852-948_L2A_T31TCJ_C_V3-0_SRE_B2.tif", #Chemin fichier en entrée
    format="GTiff",
    cutlineDSName=geojson_str,  # Passer directement le GeoJSON
    cropToCutline=True,
    dstSRS="EPSG:2154",
    xRes=resolution,
    yRes=resolution,
    dstNodata=-9999
)

<osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x7fd7dd8d63a0> >

In [15]:
test=rw.open_image('/home/onyxia/work/test.tif')
rw.get_pixel_size(test)

(10.0, -10.0)